# Import libraries

In [ ]:
import sys
import glob, os
import pandas as pd
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.offline as offline
from plotly import tools
import time

from scipy.spatial import distance
from scipy import linalg
from scipy import signal
import matplotlib.pyplot as plt

%matplotlib inline
offline.init_notebook_mode()
pd.options.display.float_format = '{:.6f}'.format
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

sys.path.insert(0, '../../scripts/asset_processor/')
# load the autoreload extension
%load_ext autoreload
# Set extension to reload modules every time before executing code
%autoreload 2

from video_asset_processor import video_asset_processor
from video_metrics import video_metrics


# Computes a list of metrics for some assets

In [ ]:
path = '../../stream/'

resolutions = [
               144, 
               240, 
               360,
               480,
               720,
               1080
              ]

attacks = [
           'watermark',
           'watermark-345x114',
           'watermark-856x856',
           'vignette',
           'low_bitrate_4',
           'low_bitrate_8',
           'black_and_white',
#            'flip_vertical',
           'rotate_90_clockwise'
          ]

seconds = 2
max_samples = 10
total_videos = 5

assets = np.random.choice(os.listdir(path + '1080p'), total_videos, replace=False)

metrics_list = ['temporal_texture',
                'temporal_gaussian',
                'temporal_gaussian_difference_threshold', 
                'temporal_dct'
               ]

metrics_df = pd.DataFrame()
init = time.time()

for asset in assets:
    original_asset = path + '1080p/' + asset
    renditions_list = [path + str(res) + 'p/' + asset for res in resolutions]
    attack_list = [path + str(res) + 'p_' + attk + '/' + asset for res in resolutions for attk in attacks]
    renditions_list += attack_list
    asset_processor = video_asset_processor(original_asset, renditions_list, metrics_list, seconds,
                                            max_samples)
    metrics = asset_processor.process()
    metrics_df = metrics_df.append(metrics)

print('That took {0:.2f} seconds to compute'.format(time.time() - init))

In [ ]:
metrics_df = metrics_df.reset_index()
metrics_df = metrics_df.drop('index', axis=1)


In [ ]:
features = [metric + '-mean' for metric in metrics_list]
columns = ['dimension', 'size', 'title', 'attack'] + features
metrics_df = metrics_df.filter(columns)
attacks = metrics_df['attack'].unique()
metrics_df.head()

In [ ]:
legit_df = metrics_df[~metrics_df['attack'].str.contains('_')]
attacks_df = metrics_df[metrics_df['attack'].str.contains('_')]
display(legit_df.head())
display(attacks_df.head())

# Feature analysis

In [ ]:
print('Legit assets description:')
display(legit_df[features].astype(float).describe())
print('Attack assets description:')
display(attacks_df[features].astype(float).describe())

In [ ]:
for feat in features: 
    data = []
    data.append(go.Box(y=attacks_df[feat], name='Attacks', boxpoints='all',
                       text=attacks_df['title'] + '-' + attacks_df['attack']))
    data.append(go.Box(y=legit_df[feat], name='Legit', boxpoints='all',
                       text=attacks_df['title'] + '-' + attacks_df['attack']))
    layout = go.Layout(
            title=feat,
            yaxis = go.layout.YAxis(title = feat),
            xaxis = go.layout.XAxis(
                title = 'Type of asset',
                )
            )
    fig = go.Figure(data=data, layout=layout)

    offline.iplot(fig)

In [ ]:
df_corr = attacks_df[features].astype(float).corr()
plt.figure(figsize=(10,10))
corr = df_corr.corr('pearson')
corr.style.background_gradient().set_precision(2)

In [ ]:
df_corr = legit_df[features].astype(float).corr()
plt.figure(figsize=(10,10))
corr = df_corr.corr('pearson')
corr.style.background_gradient().set_precision(2)